In [ ]:
import os
import re
import pandas as pd 

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#2_text'}.xlsx")
df = pd.read_excel(xname, index_col=0)
df = df[df['image']==0].reset_index(drop=True)
result_list = []
print(len(df))
for i in range(len(df)):
    print(i)
    print(df.loc[i, 'seqnum'])
    
    seqnum = df.loc[i, 'seqnum']
    cbub_code = df.loc[i, 'cbub_code']
    title = df.loc[i, "title"]
    url = df.loc[i, 'url']
    path = df.loc[i, 'path']
    text = df.loc[i, 'text']
    image = df.loc[i, 'image']
    
    if '원심판결' in text:
        appeal = 1
    else:
        appeal = 0
    
    text = re.sub(r"(-\s?\d\s?-)", ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'_{2,}', '', text)
    
    try:
        info = re.search(r'\b사 건\b(.+)', text).group(1) #사건 번호가 없으면 파일 앞 부분 잘린 것.
    except:
        continue
        
    cid = info.split()[0]
    case = re.sub(r"(\d+\w+)", "", info).strip()
    undertrial = re.findall(r'\b피 고 인\s(.+)[\n{2,}]+(.+)+', text)
    try:
        prosecutor = re.search(r'\b검 사\s(.+)', text).group(1)
    except:
        prosecutor = ''
    
    try:
        sentence = re.sub(r'\n{2,}', ' ', text[text.index('주 문')+3:text.rindex('이 유')])
        if '재판장' in text:
            honor = re.findall(r'\b재판장|판사\b(.+)', text)
            body = text[text.index('이 유'):text.rindex('재판장')]
        else:
            honor = re.search(r'\b판사 \b(.+)', text).group(1)
            body = text[text.index('이 유'):text.rindex('판사')]
    except:
        try:
            sentence = re.search('판 결 선 고.+\n{2,}(.+)', text).group(1)
            if '재판장' in text:
                honor = re.findall(r'\b재판장|판사\b(.+)', text)
                body = text[text.index(sentence)+len(sentence):text.rindex('재판장')]
            else:
                try:
                    honor = re.search(r'\b판사 \b(.+)', text).group(1)
                    body = text[text.index(sentence)+len(sentence):text.rindex('판사')]
                except:
                    honor = '' #확인해보고 파일 잘린 거면 삭제할 것.
                    body = text[text.index(sentence)+len(sentence):]
        except:
            continue # 형사 사건이 아닌데 크롤링 된 경우
    body = body.replace('이 유', '').replace('범 죄 사 실', '').replace('증거의 요지', '').replace('법령의 적용', '').replace('양형의 이유', '')
    body = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', body)
    body = re.sub("\n{2,}", "", body)
    body = re.sub(r'(\d+\.\s\d+\.\s\d+\.)(\s\d+:\d+경)?', '', body)
    body = re.sub(r'[\(\[].*?[\)\]]', '', body)
    
    new_dict={
        'seqnum': seqnum,
        'cbub_code': cbub_code,
        'title': title,
        'url': url,
        'path': path,
        'text': text, 
        'image': image,
        'appeal': appeal,
        'cid': cid,
        'case': case,
        'undertrial': undertrial,
        'prosecutor': prosecutor,
        'sentence': sentence,
        'honor': honor,
        'body': body
    }
    result_list.append(new_dict)
    
df = pd.DataFrame(result_list)        
save_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#3_columnize'}.xlsx")
excel_writer = pd.ExcelWriter(save_name, engine='xlsxwriter')
df.to_excel(excel_writer)
excel_writer.save()
print(len(df))